In [ ]:
#Import useful packages 
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os
import numpy as np
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# The dataset is composed of images from WIDER Face (http://shuoyang1213.me/WIDERFACE/) 
# and MAFA(http://www.escience.cn/people/geshiming/mafa.html)
# Once the images are downloaded from above two sites, they have been augmented to filtered out only faces, with mask and without mask

# Download train.zip from https://drive.google.com/file/d/1cW6Nmt4Vl-DJ9yZyr7dE1FmYbyZAYAZx/view?usp=sharing 
# Then upload the zipped file from Left section and unzip images for training
 #unzip -q /content/drive/train.zip 
!unzip -q "drive/My Drive/train.zip" 


In [ ]:
# The dataset is composed of images from WIDER Face (http://shuoyang1213.me/WIDERFACE/) 
# and MAFA(http://www.escience.cn/people/geshiming/mafa.html)
# Once the images are downloaded from above two sites, they have been augmented to filtered out only faces, with mask and without mask


# Download validation.zip from https://drive.google.com/file/d/1Tyd9Ods8emWB2r6XHFAiRNu8dgVJuqWv/view?usp=sharing 
# Then upload the zipped file from Left section and unzip images for validation
!unzip -q "drive/My Drive/validation.zip" 


In [ ]:
!unzip -q "drive/My Drive/Test.zip" 

In [ ]:
PATH = "/content/"

In [ ]:
# Defining directory path of training and validation 
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH,'Test')

In [ ]:
# Defining Two classes directory 
train_face_dir = os.path.join(train_dir, 'face')  
train_mask_dir = os.path.join(train_dir, 'mask') 

validation_face_dir = os.path.join(validation_dir, 'face') 
validation_mask_dir = os.path.join(validation_dir, 'mask')

test_mask_dir = os.path.join(test_dir,'Mask')
test_face_dir = os.path.join(test_dir,'Non Mask')

In [ ]:
# Calculating number of training and validation images
num_face_tr = len(os.listdir(train_face_dir))
num_mask_tr = len(os.listdir(train_mask_dir))

num_face_val = len(os.listdir(validation_face_dir))
num_mask_val = len(os.listdir(validation_mask_dir))

num_mask_test = len(os.listdir(test_mask_dir))
num_face_test=len(os.listdir(test_face_dir))

total_train = num_face_tr + num_mask_tr
total_val = num_face_val + num_mask_val
total_test = num_mask_test +num_face_test

In [ ]:
print(total_train)
print(total_val)
print(total_test)

4430
1916
100


In [ ]:
batch_size = 32
IMG_HEIGHT = 160
IMG_WIDTH = 160

In [ ]:
#ImageDataGenerator accepts the original data, randomly transforms it, and returns only the new, transformed data
train_image_generator = ImageDataGenerator(rescale=1./255) # Normalize all pixel values between 0 & 1
validation_image_generator = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# flow_from_directory Extract data from specified folder where your '2' or 'n' classes of folders are present
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')
test_data_gen = test_gen.flow_from_directory(
                                              batch_size=batch_size,
                                              directory=validation_dir,
                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                              class_mode='binary')

Found 4430 images belonging to 2 classes.
Found 1916 images belonging to 2 classes.
Found 1916 images belonging to 2 classes.


In [ ]:
# Define Input shape of images for InceptionV3 model
IMG_SHAPE = (160, 160, 3)
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

87916544/87910968 [==============================] - 0s 0us/step


In [ ]:
# Printing total number of layers of InceptionV3 model
len(base_model.layers)

311

In [ ]:
# base model architecture
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 79, 79, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 79, 79, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
# Freeze the convolutional base before you compile and train the model
base_model.trainable = False 

In [ ]:
flatten_layer = tf.keras.layers.Flatten()
flatten_layer

In [ ]:
# Flatten works by converting Matrix to single array/column
flatten_layer = tf.keras.layers.Flatten()
# Dense or fully-connected layers are used when association can exist among any feature to any other feature in data point
Dense1 = tf.keras.layers.Dense(units=500)  # default activation=None,i.e. Linear Matrix multiplication
# Batch normalization is used to apply normalization to the output of the hidden layers.
batch_normalization = tf.keras.layers.BatchNormalization()
# A dropout layer is used for regularization where you randomly set some of the dimensions of your input vector to be zero. 
dropout = tf.keras.layers.Dropout(0.2)
# Output will be a Tensor representing the input tensor, transformed by the relu activation function
relu_activation1 = tf.keras.layers.ReLU()
Dense2 = tf.keras.layers.Dense(units=128)
batch_normalization2 = tf.keras.layers.BatchNormalization()
relu_activation2 = tf.keras.layers.ReLU() # standard ReLU activation: max(x, 0)
output = tf.keras.layers.Dense(units=1,activation='sigmoid') # i.e. output = activation(dot(input, kernel) + bias)

In [ ]:
# # Now stack the feature extracted above layer by layer using a tf.keras.Sequential model
model = tf.keras.Sequential([
  base_model,
  flatten_layer,
  Dense1,
  batch_normalization,
    dropout,
relu_activation1,
  Dense2,
batch_normalization2,
relu_activation2,
    output
])

In [ ]:
# Compile the model before training it. Since there are two classes, use a 
# binary cross-entropy loss with from_logits=True since the model provides a linear output
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 3, 3, 2048)        21802784  
_________________________________________________________________
flatten_3 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               9216500   
_________________________________________________________________
batch_normalization_96 (Batc (None, 500)               2000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
re_lu_2 (ReLU)               (None, 500)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               6

In [ ]:
# steps_per_epoch : it specifies the total number of steps taken from the generator
# epochs : an integer and number of epochs we want to train our model for.
# validation_data can be an inputs and targets list
# validation_steps:only if the validation_data is a generator then only this argument can be used
# fit_generator() function first accepts a batch of the dataset, 
# then performs backpropagation on it, and then updates the weights in our model

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train / batch_size,
    epochs=70,
    validation_data=val_data_gen,
    validation_steps=total_val / batch_size
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


138/138 [==============================] - 49s 109ms/step - loss: 0.1319 - accuracy: 0.9524 - val_loss: 0.3607 - val_accuracy: 0.9092
Epoch 2/70
138/138 [==============================] - 12s 87ms/step - loss: 0.0565 - accuracy: 0.9828 - val_loss: 0.3659 - val_accuracy: 0.9092
Epoch 3/70
138/138 [==============================] - 12s 87ms/step - loss: 0.0316 - accuracy: 0.9926 - val_loss: 0.3452 - val_accuracy: 0.9050
Epoch 4/70
138/138 [==============================] - 12s 86ms/step - loss: 0.0187 - accuracy: 0.9968 - val_loss: 0.3194 - val_accuracy: 0.9160
Epoch 5/70
138/138 [==============================] - 12s 86ms/step - loss: 0.0148 - accuracy: 0.9962 - val_loss: 0.3791 - val_accuracy: 0.9144
Epoch 6/70
138/138 [==============================] - 12s 89ms/step - loss: 0.0095 - accuracy: 0.9984 - val_loss: 0.5013 - val_accuracy: 0.8956
Epoch 7/70
138/138 [==============================] - 12s 87ms/step - loss: 0.0055 - accuracy: 0.9993 - val_loss: 0.3321 - val_accuracy: 0.9092
Ep

In [ ]:
# now we need to see how it was trained
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
# testing on new data
# since the model is giving good out put now we feed test data to it
test_loss, test_acc = model.evaluate(test_data_gen)
print('test_loss:{} test_acc:{}'.format(test_loss,test_acc))
# good accuracy of 92%

60/60 [==============================] - 4s 60ms/step - loss: 1.4297 - accuracy: 0.9149
test_loss:1.429718255996704 test_acc:0.9149269461631775


In [ ]:
from google.colab import files
from keras.preprocessing import image
uploaded =  files.upload()  # this will help in uploading files from computer
for fname in uploaded.keys():
  img_path = '/content/'+fname # that means each image will be uploaded to collab content directory
  img = image.load_img(img_path,target_size=(160,160))
  # now change image to array
  images = image.img_to_array(img)
  images = np.expand_dims(images,axis=0) # to expand shape of an array
  prediction = model.predict(images)
  print(fname)
  if prediction ==0:
    print('Mask!!!!!!!!!\n')
  else:
    print('No Mask!!!!!!!!!!!!\n')

Saving nm.jpg to nm (1).jpg
nm.jpg
No Mask!!!!!!!!!!!!



In [ ]:
model.save("inception_face_mask_detection.h5")